# Notebook for cleaning `MEDICATIONS TABLE`

In [1]:
import pandas as pd
import numpy as np
 
from sqlalchemy import create_engine,text
user = 'root'
passwords = '7003890541'
host = 'localhost'
port = 3306
database = 'synthea'

engine = create_engine(f"mysql+pymysql://{user}:{passwords}@{host}:{port}/{database}")
def sql(query , engine = engine):
    return pd.read_sql(query,engine)


def select(table_name):
    return sql(f'select * from {table_name} limit 5')


def run(query):
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [2]:
select('medications')

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2010-05-05T00:26:23Z,2011-04-30T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,Etonogestrel 68 MG Drug Implant,677.08,0.0,12,8124.96,NaN,None
1,2011-04-30T00:26:23Z,2012-04-24T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,389221,Etonogestrel 68 MG Drug Implant,624.09,0.0,12,7489.08,NaN,None
2,2012-04-24T00:26:23Z,2013-04-19T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7253a9f9-6f6d-429a-926a-7b1d424eae3f,748856,Yaz 28 Day Pack,43.32,0.0,12,519.84,NaN,None
3,2011-05-13T12:58:08Z,2011-05-27T12:58:08Z,10339b10-3cd1-4ac3-ac13-ec26728cb592,d47b3510-2895-3b70-9897-342d681c769d,e1ab4933-07a1-49f0-b4bd-05500919061d,313782,Acetaminophen 325 MG Oral Tablet,8.14,0.0,1,8.14,10509002.0,Acute bronchitis (disorder)
4,2011-12-08T15:02:18Z,2011-12-22T15:02:18Z,1d604da9-9a81-4ba9-80c2-de3375d59b40,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,792fae81-a007-44b0-8221-46953737b089,562251,Amoxicillin 250 MG / Clavulanate 125 MG Oral T...,11.91,0.0,1,11.91,444814009.0,Viral sinusitis (disorder)


In [3]:
query = '''
    alter table medications
    drop column description,
    drop column reasoncode;
'''
run(query)

In [4]:
select('medications')

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONDESCRIPTION
0,2010-05-05T00:26:23Z,2011-04-30T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,677.08,0.0,12,8124.96,None
1,2011-04-30T00:26:23Z,2012-04-24T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,389221,624.09,0.0,12,7489.08,None
2,2012-04-24T00:26:23Z,2013-04-19T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7253a9f9-6f6d-429a-926a-7b1d424eae3f,748856,43.32,0.0,12,519.84,None
3,2011-05-13T12:58:08Z,2011-05-27T12:58:08Z,10339b10-3cd1-4ac3-ac13-ec26728cb592,d47b3510-2895-3b70-9897-342d681c769d,e1ab4933-07a1-49f0-b4bd-05500919061d,313782,8.14,0.0,1,8.14,Acute bronchitis (disorder)
4,2011-12-08T15:02:18Z,2011-12-22T15:02:18Z,1d604da9-9a81-4ba9-80c2-de3375d59b40,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,792fae81-a007-44b0-8221-46953737b089,562251,11.91,0.0,1,11.91,Viral sinusitis (disorder)


In [5]:
query = '''
    alter table medications
    rename column totalcost to TOTAL_COST,
    rename column REASONDESCRIPTION to REASON;
    '''
run(query)

In [6]:


query = '''
    select column_name from information_schema.columns
    where table_name = 'medications' and
    table_schema = database();
'''
sql(query)
medications_columns = [c[0] for c in sql(query).values.tolist()]

In [7]:
for col in medications_columns:
    query = f'''
        update medications
        set {col} = lower({col})
    '''
    run(query)

In [8]:
select('medications')

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTAL_COST,REASON
0,2010-05-05t00:26:23z,2011-04-30t00:26:23z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,677.08,0.0,12,8124.96,None
1,2011-04-30t00:26:23z,2012-04-24t00:26:23z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,389221,624.09,0.0,12,7489.08,None
2,2012-04-24t00:26:23z,2013-04-19t00:26:23z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7253a9f9-6f6d-429a-926a-7b1d424eae3f,748856,43.32,0.0,12,519.84,None
3,2011-05-13t12:58:08z,2011-05-27t12:58:08z,10339b10-3cd1-4ac3-ac13-ec26728cb592,d47b3510-2895-3b70-9897-342d681c769d,e1ab4933-07a1-49f0-b4bd-05500919061d,313782,8.14,0.0,1,8.14,acute bronchitis (disorder)
4,2011-12-08t15:02:18z,2011-12-22t15:02:18z,1d604da9-9a81-4ba9-80c2-de3375d59b40,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,792fae81-a007-44b0-8221-46953737b089,562251,11.91,0.0,1,11.91,viral sinusitis (disorder)


In [9]:
for col in medications_columns:
    query = f'''
        alter table medications
        rename column {col} to {col.lower()}
    '''
    run(query)

In [10]:
select('medications')

,start,stop,patient,payer,encounter,code,base_cost,payer_coverage,dispenses,total_cost,reason
0,2010-05-05t00:26:23z,2011-04-30t00:26:23z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,677.08,0.0,12,8124.96,None
1,2011-04-30t00:26:23z,2012-04-24t00:26:23z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,389221,624.09,0.0,12,7489.08,None
2,2012-04-24t00:26:23z,2013-04-19t00:26:23z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7253a9f9-6f6d-429a-926a-7b1d424eae3f,748856,43.32,0.0,12,519.84,None
3,2011-05-13t12:58:08z,2011-05-27t12:58:08z,10339b10-3cd1-4ac3-ac13-ec26728cb592,d47b3510-2895-3b70-9897-342d681c769d,e1ab4933-07a1-49f0-b4bd-05500919061d,313782,8.14,0.0,1,8.14,acute bronchitis (disorder)
4,2011-12-08t15:02:18z,2011-12-22t15:02:18z,1d604da9-9a81-4ba9-80c2-de3375d59b40,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,792fae81-a007-44b0-8221-46953737b089,562251,11.91,0.0,1,11.91,viral sinusitis (disorder)


In [11]:
query = """
UPDATE medications
SET start = STR_TO_DATE(
    TRIM(REPLACE(REPLACE(REPLACE(start, '"', ''), '\r', ''), '\n', '')),
    '%Y-%m-%dt%H:%i:%sz'
);
"""
run(query)

In [12]:
query = """
UPDATE medications
SET stop = STR_TO_DATE(
    TRIM(REPLACE(REPLACE(REPLACE(stop, '"', ''), '\r', ''), '\n', '')),
    '%Y-%m-%dt%H:%i:%sz'
);
"""
run(query)

In [13]:
query = """
ALTER TABLE medications
MODIFY start datetime,
modify stop datetime;
"""
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [14]:
select('medications')

,start,stop,patient,payer,encounter,code,base_cost,payer_coverage,dispenses,total_cost,reason
0,2010-05-05 00:26:23,2011-04-30 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,677.08,0.0,12,8124.96,None
1,2011-04-30 00:26:23,2012-04-24 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,389221,624.09,0.0,12,7489.08,None
2,2012-04-24 00:26:23,2013-04-19 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7253a9f9-6f6d-429a-926a-7b1d424eae3f,748856,43.32,0.0,12,519.84,None
3,2011-05-13 12:58:08,2011-05-27 12:58:08,10339b10-3cd1-4ac3-ac13-ec26728cb592,d47b3510-2895-3b70-9897-342d681c769d,e1ab4933-07a1-49f0-b4bd-05500919061d,313782,8.14,0.0,1,8.14,acute bronchitis (disorder)
4,2011-12-08 15:02:18,2011-12-22 15:02:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,792fae81-a007-44b0-8221-46953737b089,562251,11.91,0.0,1,11.91,viral sinusitis (disorder)


In [15]:
query = '''
    alter table medications
    rename column code to medicine_code;
    '''
run(query)

In [16]:
select('medications')

,start,stop,patient,payer,encounter,medicine_code,base_cost,payer_coverage,dispenses,total_cost,reason
0,2010-05-05 00:26:23,2011-04-30 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,677.08,0.0,12,8124.96,None
1,2011-04-30 00:26:23,2012-04-24 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,389221,624.09,0.0,12,7489.08,None
2,2012-04-24 00:26:23,2013-04-19 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7253a9f9-6f6d-429a-926a-7b1d424eae3f,748856,43.32,0.0,12,519.84,None
3,2011-05-13 12:58:08,2011-05-27 12:58:08,10339b10-3cd1-4ac3-ac13-ec26728cb592,d47b3510-2895-3b70-9897-342d681c769d,e1ab4933-07a1-49f0-b4bd-05500919061d,313782,8.14,0.0,1,8.14,acute bronchitis (disorder)
4,2011-12-08 15:02:18,2011-12-22 15:02:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,792fae81-a007-44b0-8221-46953737b089,562251,11.91,0.0,1,11.91,viral sinusitis (disorder)


In [17]:


query = '''
    select column_name from information_schema.columns
    where table_name = 'medications' and
    table_schema = database();
'''
sql(query)
medications_columns = [c[0] for c in sql(query).values.tolist()]

In [18]:
for col in medications_columns:
    query = f'''
        select sum({col} is null) as {col}_null from medications
        having {col}_null > 0;
    '''
    display(sql(query))

,start_null


,stop_null
0,1895.0


,patient_null


,payer_null


,encounter_null


...

,base_cost_null


,payer_coverage_null


,dispenses_null


,total_cost_null


,reason_null
0,11117.0


In [19]:
query = '''
    update medications
    set reason = 'unknown' where reason is null;
'''
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [20]:
for col in medications_columns:
    query = f'''
        select sum({col} is null) as {col}_null from medications
        having {col}_null > 0;
    '''
    display(sql(query))

,start_null


,stop_null
0,1895.0


,patient_null


,payer_null


,encounter_null


...

,base_cost_null


,payer_coverage_null


,dispenses_null


,total_cost_null


,reason_null


In [21]:
select('medications')

,start,stop,patient,payer,encounter,medicine_code,base_cost,payer_coverage,dispenses,total_cost,reason
0,2010-05-05 00:26:23,2011-04-30 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,677.08,0.0,12,8124.96,unknown
1,2011-04-30 00:26:23,2012-04-24 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,389221,624.09,0.0,12,7489.08,unknown
2,2012-04-24 00:26:23,2013-04-19 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7253a9f9-6f6d-429a-926a-7b1d424eae3f,748856,43.32,0.0,12,519.84,unknown
3,2011-05-13 12:58:08,2011-05-27 12:58:08,10339b10-3cd1-4ac3-ac13-ec26728cb592,d47b3510-2895-3b70-9897-342d681c769d,e1ab4933-07a1-49f0-b4bd-05500919061d,313782,8.14,0.0,1,8.14,acute bronchitis (disorder)
4,2011-12-08 15:02:18,2011-12-22 15:02:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,792fae81-a007-44b0-8221-46953737b089,562251,11.91,0.0,1,11.91,viral sinusitis (disorder)
